In [0]:
# Loadind data from S3
from pyspark.sql import functions as F
filepath = "s3://full-stack-bigdata-datasets/Big_Data/youtube_playlog.csv"
playlog = (spark.read.format('csv')\
            .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load(filepath))
playlog.show(5)

In [0]:
# Print out the DataFrame schema
playlog.printSchema()


In [0]:
playlog.describe().toPandas()


In [0]:
# Couning  the missing values for each column putting the result in a pandas DataFrame and printing it out
import pandas as pd

pd.DataFrame.from_dict(
  {c: playlog.filter(playlog[c].isNull()).count() for c in playlog.columns},
  orient='index', columns=['missing values']
).T

In [0]:
# Checking if playlog without duplicates has the same number of rows as the original
playlog.count() == playlog.dropDuplicates().count()


In [0]:
# Counting the duplicates
(playlog.count() - playlog.dropDuplicates().count())


In [0]:
# Ordering the dataframe by ascending timestamp and showing the first 5 rows
playlog.orderBy('timestamp').show(5)

In [0]:
# Counting the number of rows with a negative timestamp
playlog.where(playlog['timestamp'] < 0).count()

In [0]:
# Collecting the problematic rows
playlog.where(playlog['timestamp'] < 0).collect()

In [0]:
# Removing the row with a neagtive timestamp
playlog_processed = (playlog.dropDuplicates() \
                       .filter(~(playlog.timestamp < 0)))
playlog_processed.count()

In [0]:
# Saving the processed DataFrame to S3
output_path = 's3://full-stack-bigdata-datasets/Big_Data/playlog_processed_student.parquet'

playlog_processed.write \
  .parquet(output_path, mode='overwrite')